In [193]:
import pandas as pd
import numpy as np

In [194]:
from sklearn.linear_model import LinearRegression, Ridge, HuberRegressor

In [195]:
from lightgbm import LGBMRegressor

In [196]:
import sys
sys.path.append('../')
from preprocessing.preprocessing import preprocessing

In [197]:
#gte = gte = pd.read_csv("../features/gte_features_w8_prp50.csv")
#gte.Date = pd.to_datetime(gte.Date)

In [198]:
path_train = '../dataset/prediction/val/'
path_test =  '../dataset/prediction/test/'

lgb_std_train = pd.read_csv(path_train + "lgb_std_val.csv")
lgb_std_test = pd.read_csv(path_test + "lgb_std_test.csv")

lgb_cls_train = pd.read_csv(path_train + "lgb_cls_val.csv")
lgb_cls_test = pd.read_csv(path_test + "lgb_cls_test.csv")

cat_std_train = pd.read_csv(path_train + "cat_std_val.csv")
cat_std_test = pd.read_csv(path_test + "cat_std_test.csv")

cat_cls_train = pd.read_csv(path_train + "cat_cls_val.csv")
cat_cls_test = pd.read_csv(path_test + "cat_cls_test.csv")

#xgb_train = pd.read_csv(path_train + "xgb_incremental_val.csv")
#xgb_test = pd.read_csv(path_test + "xgb_incremental_test.csv")

lin_reg_train = pd.read_csv(path_train + "linear_reg_val.csv")
lin_reg_test = pd.read_csv(path_test + "linear_reg_test.csv")

In [199]:
train = pd.read_csv("../dataset/original/train.csv")
test = pd.read_csv("../dataset/original/x_test.csv")
df = preprocessing(train, test, useTest=False)

../preprocessing/preprocessing.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017['order'] = df_2017.sort_values(['sku', 'Date']).groupby('sku').cumcount()
../preprocessing/preprocessing.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['order'] = df_2018.sort_values(['sku', 'Date']).groupby('sku').cumcount()


In [200]:
prediction_train = lgb_cls_train.merge(lgb_std_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_train = prediction_train.merge(cat_std_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_train = prediction_train.merge(cat_cls_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
#prediction_train = prediction_train.merge(xgb_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_train = prediction_train.merge(lin_reg_train, how='left', on=['Date', 'sku', 'target', 'real_target'])

prediction_train.Date = pd.to_datetime(prediction_train.Date)
prediction_train = prediction_train.merge(df[['Date', 'sku', 'sales w-1']], how='left', on=['Date', 'sku'])
#prediction_train = prediction_train.merge(gte, how='left', on=['Date', 'sku', 'target', 'real_target'])

In [201]:
#prediction_train = prediction_train[prediction_train.log_prediction_lgb_cls != 0.0]

In [202]:
#prediction_train = prediction_train.merge(df[['Date', 'sku', 'sales w-1']], how='left')

In [203]:
prediction_test = lgb_cls_test.merge(lgb_std_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_test = prediction_test.merge(cat_std_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_test = prediction_test.merge(cat_cls_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
#prediction_test = prediction_test.merge(xgb_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_test = prediction_test.merge(lin_reg_test, how='left', on=['Date', 'sku', 'target', 'real_target'])

prediction_test.Date = pd.to_datetime(prediction_test.Date)
prediction_test = prediction_test.merge(df[['Date', 'sku', 'sales w-1']], how='left')
#prediction_test = prediction_test.merge(gte, how='left', on=['Date', 'sku', 'target', 'real_target'])

In [204]:
mask = (prediction_train.Date=='2017-01-07')
prediction_train = prediction_train.drop(prediction_train[mask].index)

In [205]:
prediction_test = prediction_test[~prediction_test.prediction_linear_reg.isna()]

In [206]:
cols = ['log_prediction_lgb_cls',
        'log_prediction_lgb_std',
        'log_prediction_cat_std',
        'log_prediction_cat_cls',
        'log_prediction_linear_reg',
        #'prediction',
        #'sales w-1'
       ]
#gte_cols = [x for x in prediction_train.columns if 'gte' in x]
#cols = cols + gte_cols
cols

['log_prediction_lgb_cls',
 'log_prediction_lgb_std',
 'log_prediction_cat_std',
 'log_prediction_cat_cls',
 'log_prediction_linear_reg']

In [207]:
#prediction_train = prediction_train[prediction_train.Date > '2018-01-06']

In [208]:
reg = LinearRegression()
reg.fit(prediction_train[cols], prediction_train.target)

LinearRegression()

In [34]:
lgb = LGBMRegressor(n_estimators=800)
train = prediction_train[prediction_train.Date < '2019-05-11']
val = prediction_train[prediction_train.Date > '2019-05-11']
lgb.fit(train[cols], train.target, eval_set=[(val[cols], val.target)], eval_metric='mape', 
        early_stopping_rounds=200, categorical_feature=['sku'])

[1]	valid_0's mape: 0.0726443	valid_0's l2: 0.816398
Training until validation scores don't improve for 200 rounds
[2]	valid_0's mape: 0.0656633	valid_0's l2: 0.667293
[3]	valid_0's mape: 0.0593303	valid_0's l2: 0.545613
[4]	valid_0's mape: 0.0536922	valid_0's l2: 0.447294
[5]	valid_0's mape: 0.0485863	valid_0's l2: 0.367184
[6]	valid_0's mape: 0.04402	valid_0's l2: 0.301863
[7]	valid_0's mape: 0.0400147	valid_0's l2: 0.249409
[8]	valid_0's mape: 0.0363985	valid_0's l2: 0.206291
[9]	valid_0's mape: 0.033184	valid_0's l2: 0.17178
[10]	valid_0's mape: 0.0302714	valid_0's l2: 0.143612
[11]	valid_0's mape: 0.0276946	valid_0's l2: 0.120625
[12]	valid_0's mape: 0.0253975	valid_0's l2: 0.101897
[13]	valid_0's mape: 0.0233389	valid_0's l2: 0.0866809
[14]	valid_0's mape: 0.0215699	valid_0's l2: 0.0743235
[15]	valid_0's mape: 0.0199702	valid_0's l2: 0.0640683
[16]	valid_0's mape: 0.0186286	valid_0's l2: 0.0560211
[17]	valid_0's mape: 0.0173992	valid_0's l2: 0.0491514
[18]	valid_0's mape: 0.01638

[215]	valid_0's mape: 0.00964797	valid_0's l2: 0.0201908
[216]	valid_0's mape: 0.00964191	valid_0's l2: 0.0201664
[217]	valid_0's mape: 0.00963266	valid_0's l2: 0.0201376
[218]	valid_0's mape: 0.00963331	valid_0's l2: 0.020137
[219]	valid_0's mape: 0.00963963	valid_0's l2: 0.0201424
[220]	valid_0's mape: 0.00964268	valid_0's l2: 0.0201415
[221]	valid_0's mape: 0.0096432	valid_0's l2: 0.0201482
[222]	valid_0's mape: 0.00964272	valid_0's l2: 0.0201422
[223]	valid_0's mape: 0.00964663	valid_0's l2: 0.0201515
[224]	valid_0's mape: 0.00965966	valid_0's l2: 0.0201667
[225]	valid_0's mape: 0.00965235	valid_0's l2: 0.0201383
[226]	valid_0's mape: 0.00965156	valid_0's l2: 0.0201332
[227]	valid_0's mape: 0.00966217	valid_0's l2: 0.0201531
[228]	valid_0's mape: 0.00966275	valid_0's l2: 0.020154
[229]	valid_0's mape: 0.00965827	valid_0's l2: 0.0201568
[230]	valid_0's mape: 0.00965868	valid_0's l2: 0.0201478
[231]	valid_0's mape: 0.00965576	valid_0's l2: 0.0201368
[232]	valid_0's mape: 0.00965694	v

LGBMRegressor(n_estimators=800)

In [35]:
prediction_test['ensemble'] = lgb.predict(prediction_test[cols], num_iteration=lgb.best_iteration_)

In [209]:
prediction_test['ensemble'] = reg.predict(prediction_test[cols]) / sum(reg.coef_)
prediction_test['ensemble'] = np.expm1(prediction_test.ensemble)

In [210]:
prediction_test.ensemble = round(prediction_test.ensemble)

In [211]:
from metrics.MAPE import MAPE

In [212]:
MAPE(prediction_test[~prediction_test.target.isna()].real_target, prediction_test[~prediction_test.target.isna()].ensemble)

5.2436890933152736

In [213]:
prediction_train[cols].columns

Index(['log_prediction_lgb_cls', 'log_prediction_lgb_std',
       'log_prediction_cat_std', 'log_prediction_cat_cls',
       'log_prediction_linear_reg'],
      dtype='object')

In [214]:
reg.coef_

array([0.0328162 , 0.16261115, 0.03310335, 0.10411719, 0.67143726])

In [129]:
dates = prediction_train.Date.sort_values().drop_duplicates()

In [71]:
mapes = {}
for d in dates:
    mask = prediction_train.Date == d
    mapes[d] = MAPE(prediction_train[mask].real_target, prediction_train[mask].prediction_cat_std)
mapes

{Timestamp('2016-12-24 00:00:00'): 14.415419882614295,
 Timestamp('2016-12-31 00:00:00'): 45.562330082885005,
 Timestamp('2017-01-07 00:00:00'): 29.664381922703964,
 Timestamp('2017-01-14 00:00:00'): 27.541497877295473,
 Timestamp('2017-01-21 00:00:00'): 8.17772527419612,
 Timestamp('2017-01-28 00:00:00'): 6.301667653505562,
 Timestamp('2017-02-04 00:00:00'): 7.296266085058717,
 Timestamp('2017-02-11 00:00:00'): 10.796785933974483,
 Timestamp('2017-02-18 00:00:00'): 7.505157553437786,
 Timestamp('2017-02-25 00:00:00'): 4.86994972510147,
 Timestamp('2017-03-04 00:00:00'): 11.052292530335848,
 Timestamp('2017-03-11 00:00:00'): 8.44162649600763,
 Timestamp('2017-03-18 00:00:00'): 5.199012882941633,
 Timestamp('2017-03-25 00:00:00'): 17.919341595070225,
 Timestamp('2017-04-01 00:00:00'): 12.064057094624292,
 Timestamp('2017-04-08 00:00:00'): 10.196188944728227,
 Timestamp('2017-04-15 00:00:00'): 15.817711369351656,
 Timestamp('2017-04-22 00:00:00'): 14.63200076824852,
 Timestamp('2017-04-2